In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7000)])
    except RuntimeError as e:
        print(e)

In [ ]:
%reset
%matplotlib inline

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as img

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, MaxPooling2D,Conv2D, GaussianNoise, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

train_dir       = '/home/workspace/user-workspace/crack_data_full_8_2/train'
validation_dir  = '/home/workspace/user-workspace/crack_data_full_8_2/validation'


# Augmentation
train_datagen = ImageDataGenerator(width_shift_range=0.1,
                                        height_shift_range=0.1,
                                        vertical_flip =True,
                                        horizontal_flip =True,
                                        rotation_range =10,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                       )
validation_datagen = ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(
    train_dir,                    # target directory
    classes=['0', '1'],                               
    target_size=(75, 75),     
    batch_size=30,            
                              
    class_mode='binary')      
                              
                                            
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    classes=['0', '1'],
    target_size=(75, 75),
    batch_size=32,
    class_mode='binary')


## Pretrained Network
with tf.device('/device:GPU:0'):

    model_base = InceptionResNetV2(weights='imagenet',
                       include_top=False,
                       input_shape=(75, 75, 3))

    model_base.trainable = False  # Convolution Layer 완전 동결

    model = Sequential()

    model.add(model_base)

    model.add(Flatten(input_shape=(1536,)))

    model.add(Dense(128))
    model.add(GaussianNoise(0.1))
    model.add(Activation('relu'))

    model.add(BatchNormalization())
    
    model.add(Dense(1,
                    activation='sigmoid'))
    

    model.summary()

    model.compile(optimizer=Adam(learning_rate=2e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(train_generator,
                        steps_per_epoch=375,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=88,
                        verbose=1)


    fine_tune_at = 10

    model_base.trainable = True
    
    for layer in model_base.layers[-fine_tune_at:]:
        layer.trainable =  False

    

    model.compile(optimizer=Adam(learning_rate=2e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    
    checkpoint_filepath = '/home/workspace/user-workspace/pku/checkpoint_dir/checkpoint'
    model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, 
                                                 save_weights_only=True,
                                                 monitor='val_accuracy',
                                                 mode='max',
                                                 save_best_only=True,
                                                 verbose=1)

#   ## 재학습 진행
    history = model.fit(train_generator,
                        steps_per_epoch=375,
                        epochs=30,
                        validation_data=validation_generator,
                        validation_steps=88,
                        verbose=1,
                        callbacks=[model_checkpoint_callback])

    model.load_weights(checkpoint_filepath)

train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(train_acc, 'bo', color='r', label='training accuracy')
plt.plot(val_acc, 'b', color='b', label='validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

plt.plot(train_loss, 'bo', color='r', label='training loss')
plt.plot(val_loss, 'b', color='b', label='validation loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

model.save('/home/workspace/user-workspace/pku/model_save/crack_inception_resenet_full_22.h5')